<a href="https://colab.research.google.com/github/Titoffifee/CP_Dagestan/blob/main/transport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import recall_score, make_scorer, f1_score

df = pd.read_csv('drive/MyDrive/transport/train.csv')

In [3]:
y_labels = ['Погибло', 'Погибло детей', 'Ранено', 'Ранено детей']
delete = ['id', 'Километр', 'Метр', 'Дом', 'Дата']

In [4]:
train, y0 = df.drop(delete + y_labels, axis=1), df[y_labels]

In [5]:
train['Вид ДТП'] = train['Вид ДТП'].fillna('Неизвестно')
train['Улица'] = train['Улица'].fillna('На шоссе')
train['Дорога'] = train['Дорога'].fillna('В городе')

In [6]:
from datetime import time

# 7:50-9:50 и 18:00-20:20 - час пик
# 23:00 - 6:00 - ночь

ph0, ph1, ph2, ph3 = time(7, 50, 0), time(9, 50, 0), time(18, 0, 0), time(20, 20, 0)
night0, night1 = time(23, 0, 0), time(6, 0, 0)

def GetGoodTime(tm):
  ans = pd.Series([0] * 2, index=['Час пик', 'Ночь'])
  tm = [int(i) for i in tm.split(':')]
  t = time(hour=tm[0], minute=tm[1], second=tm[2])
  if ph0 <= t <= ph1 or ph2 <= t <= ph3:
    ans['Час пик'] = 1
  if t >= night0 or t <= night1:
    ans['Ночь'] = 1
  return ans

train_time = train['Время'].apply(GetGoodTime)
train = train.drop(['Время'], axis=1)
train = pd.concat([train, train_time], axis=1)

In [7]:
test = pd.read_csv('drive/MyDrive/transport/test.csv')
test = test.drop(delete, axis=1)
test['Вид ДТП'] = test['Вид ДТП'].fillna('Неизвестно')
test['Улица'] = test['Улица'].fillna('На шоссе')
test['Дорога'] = test['Дорога'].fillna('В городе')
test_time = test['Время'].apply(GetGoodTime)
test = test.drop(['Время'], axis=1)
test = pd.concat([test, test_time], axis=1)

In [94]:
obj = ['Место', 'Вид ДТП', 'Улица', 'Дорога']
max_value = 9
model = CatBoostClassifier(silent=True, cat_features=obj, random_seed=0,
                              loss_function='Logloss', auto_class_weights='Balanced')
index_name = dict()
index_name_list = []
for name in y_labels:
  index_name[name] = [name + str(i) for i in range(1, max_value+1)]
  index_name_list += index_name[name]

In [ ]:
def CreateY(ser):
  ans = pd.Series(dtype=int)
  for name in y_labels:
    r = pd.Series([0] * max_value, index=index_name[name], dtype=int)
    for i in range(1, ser[name] + 1):
      r[name + str(i)] = 1
    ans = pd.concat([ans, r])
  return ans
y = y0.apply(CreateY, axis=1)
y

In [95]:
p = pd.read_csv('drive/MyDrive/transport/solve.csv')
new_columns = p[y_labels].apply(CreateY, axis=1)
p = pd.concat([p['id'], new_columns], axis=1)

In [ ]:
for name in y_labels:
  for i in range(1, max_value + 1):
    ind, last_ind = name + str(i), name + str(i-1)
    if i == 1:
      data_train = train
      data_test = test
    else:
      data_train = pd.concat([train, y[last_ind]], axis=1)
      data_test = pd.concat([test, p[last_ind]], axis=1)
    print(ind, 'WORK')
    if len(y[ind].unique()) == 1:
      break
    model.fit(data_train, y[ind])
    p[ind] = model.predict(data_test)
  print(name, 'OK')
p

In [92]:
def CreateRes(ser):
  ans = pd.Series([0] * 4, index=y_labels)
  for name in y_labels:
    r = 0
    for i in range(1, max_value + 1):
      if ser[name + str(i)] == 1:
        r = i
    ans[name] = r
  return ans

In [97]:
col = p[index_name_list].apply(CreateRes, axis=1)
p = pd.concat([p['id'], col], axis=1)
p.to_csv('drive/MyDrive/transport/ans.csv', index=False)